<!-- @format -->

The eventual goal is to update a parquet file with generation data from the French grid.
Once I have made a first step, I will map out a bunch of steps.


In [1]:
import os
import requests 

In [2]:
r = requests.post(
    "https://digital.iservices.rte-france.com/token/oauth/",
    headers={
        "Authorization": f'Basic {os.environ["RTE_FRANCE_BASE64"]}',
        "Content-Type": "application/x-www-form-urlencoded",
    },
)

In [3]:
token = r.json()["access_token"]

In [4]:
r_data = requests.get(
    "https://digital.iservices.rte-france.com/open_api/actual_generation/v1/generation_mix_15min_time_scale/",
    headers={
        "Host": "digital.iservices.rte-france.com",
        "Authorization": f"Bearer {token}",
    },
)

In [5]:
r_data.json()

{'generation_mix_15min_time_scale': [{'start_date': '2023-08-17T00:00:00+02:00',
   'end_date': '2023-08-18T00:00:00+02:00',
   'production_type': 'BIOENERGY',
   'production_subtype': 'TOTAL',
   'values': [{'start_date': '2023-08-17T00:00:00+02:00',
     'end_date': '2023-08-17T00:15:00+02:00',
     'updated_date': '2023-08-16T20:49:40+02:00',
     'value': 0},
    {'start_date': '2023-08-17T00:15:00+02:00',
     'end_date': '2023-08-17T00:30:00+02:00',
     'updated_date': '2023-08-16T20:49:40+02:00',
     'value': 0},
    {'start_date': '2023-08-17T00:30:00+02:00',
     'end_date': '2023-08-17T00:45:00+02:00',
     'updated_date': '2023-08-16T20:49:40+02:00',
     'value': 0},
    {'start_date': '2023-08-17T00:45:00+02:00',
     'end_date': '2023-08-17T01:00:00+02:00',
     'updated_date': '2023-08-16T20:49:40+02:00',
     'value': 0},
    {'start_date': '2023-08-17T01:00:00+02:00',
     'end_date': '2023-08-17T01:15:00+02:00',
     'updated_date': '2023-08-16T20:49:40+02:00',
    